In [1]:
try:
    import keras
except:
    !pip install keras

Using TensorFlow backend.
/home/liboxian/anaconda3/envs/deepfacelab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/liboxian/anaconda3/envs/deepfacelab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/liboxian/anaconda3/envs/deepfacelab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [2]:
from pathlib import Path

import tensorflow as tf
tf_session = tf.Session()
from keras import backend as K
K.set_session(tf_session)

from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

import numpy as np
import pandas as pd
from sklearn.externals import joblib

# Local library with model definitions for training and generating
from models import Generator, create_training_model

/home/liboxian/anaconda3/envs/deepfacelab/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Load Input

In [3]:
# Settings

# Percent of samples to use for training, might be necessary if you're running out of memory
sample_size = 1

# The latent dimension of the LSTM
latent_dim = 2048

# Number of epochs to train for
epochs = 10

root_path = Path('../../..')
input_path = root_path / 'input'
poem_path = input_path / 'poems'
haiku_path = poem_path / 'haikus.csv'

name = 'train_file'
output_dir = Path('output_%s' % name)
output_dir.mkdir()

In [4]:
df = pd.read_csv(str(haiku_path))
df = df.sample(frac=sample_size)
df

,0,1,2,source,0_syllables,1_syllables,2_syllables
39745,my uncle HAD to,have a brother who then had,kids what a loser,twaiku,5,7,5
82611,Hating on the Browns,isn't gonna get rid of,your man tits Jason,twaiku,5,7,5
73965,i'm sorry but if,this happens i'll stop watching,college volleyball,twaiku,5,7,5
68681,People gonna treat,you how you allow them to,treat you everytime,twaiku,5,7,5
132423,Patrick we should start,working out Me you should start,minding ur business,twaiku,5,7,5
...,...,...,...,...,...,...,...
50742,Championship game,tomorrow WE ABOUT TO,GET THAT W,twaiku,5,7,5
40233,it is okay if,you ignoring me but i,want you notice me,twaiku,5,7,5
87743,imagine being,here from the beginning at,witnessing them i,twaiku,5,7,5
93534,What a healing and,deliverance praise Indeed,our God is good,twaiku,5,"6,7","4,5"


# Format Input for Training

In [5]:
# Duplicate lines with ambiguous syllable counts
# (syllable counts where there is a comma because
# multiple pronounciations are acceptable)

lines = set([0, 1, 2])

for i in range(3):
    lines.remove(i)
    df = df[[
        '0', '1', '2',
        #'1_syllables', '2_syllables'
    ] + ['%s_syllables' % j for j in lines]].join(
        df['%s_syllables' % i].str.split(
            ',', expand=True
        ).stack(-1).reset_index(
            level=1, drop=True
        ).rename('%s_syllables' % i)
    ).drop_duplicates()
    lines.add(i)

df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
...,...,...,...,...,...,...
143132,I'm not asking did,you say it nor clarify,what you said neither,5,7,5
143133,You are truly a,moron or a liar I'm,inclined to think both,5,7,5
143134,Ain't no selfie on,this earth that's gonna make me,like Theresa May,5,7,5
143135,is doing a great,job turning Independents,into Democrats,5,7,5


In [7]:
# Drop samples that are longer that the 99th percentile of length

max_line_length = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_length) & 
    (df['1'].str.len() <= max_line_length) & 
    (df['2'].str.len() <= max_line_length)
].copy()
df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
...,...,...,...,...,...,...
143132,I'm not asking did,you say it nor clarify,what you said neither,5,7,5
143133,You are truly a,moron or a liar I'm,inclined to think both,5,7,5
143134,Ain't no selfie on,this earth that's gonna make me,like Theresa May,5,7,5
143135,is doing a great,job turning Independents,into Democrats,5,7,5


In [8]:
# Pad the lines to the max line length with new lines
for i in range(3):
    # For input, duplicate the first character
    # TODO - Why?
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_length+2, 'right', '\n')
    
    # 
    #df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    
    # TODO - trying to add the next line's first character before the line breaks
    if i == 2: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_length+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_length+2, 'right', '\n')
    
max_line_length += 2

df

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
0,Memorial Day --,a shadow for each,white cross,5,5,2,MMemorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Memorial Day --\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a shadow for each\nw\n\n\n\n\n\n\n\n\n\n\n\n\n...,wwhite cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,white cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,3,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,4,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
...,...,...,...,...,...,...,...,...,...,...,...,...
143132,I'm not asking did,you say it nor clarify,what you said neither,5,7,5,II'm not asking did\n\n\n\n\n\n\n\n\n\n\n\n\n\...,I'm not asking did\n \n\n\n\n\n\n\n\n\n\n\n\n\...,you say it nor clarify\n\n\n\n\n\n\n\n\n\n\n...,you say it nor clarify\nw\n\n\n\n\n\n\n\n\n\n...,wwhat you said neither\n\n\n\n\n\n\n\n\n\n\n\n...,what you said neither\n\n\n\n\n\n\n\n\n\n\n\n\...
143133,You are truly a,moron or a liar I'm,inclined to think both,5,7,5,YYou are truly a\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,You are truly a\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n...,moron or a liar I'm\n\n\n\n\n\n\n\n\n\n\n\n\...,moron or a liar I'm\ni\n\n\n\n\n\n\n\n\n\n\n\...,iinclined to think both\n\n\n\n\n\n\n\n\n\n\n\...,inclined to think both\n\n\n\n\n\n\n\n\n\n\n\n...
143134,Ain't no selfie on,this earth that's gonna make me,like Theresa May,5,7,5,AAin't no selfie on\n\n\n\n\n\n\n\n\n\n\n\n\n\...,Ain't no selfie on\n \n\n\n\n\n\n\n\n\n\n\n\n\...,this earth that's gonna make me\n\n\n\n\n\n\...,this earth that's gonna make me\nl\n\n\n\n\n\...,llike Theresa May\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,like Theresa May\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
143135,is doing a great,job turning Independents,into Democrats,5,7,5,iis doing a great\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,is doing a great\n \n\n\n\n\n\n\n\n\n\n\n\n\n\...,job turning Independents\n\n\n\n\n\n\n\n\n\n...,job turning Independents\ni\n\n\n\n\n\n\n\n\n...,iinto Democrats\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,into Democrats\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [9]:
inputs = df[['0_in', '1_in', '2_in']].values

tokenizer = Tokenizer(filters='', char_level=True)
tokenizer.fit_on_texts(inputs.flatten())
n_tokens = len(tokenizer.word_counts) + 1

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
    tokenizer.texts_to_sequences(inputs[:,i]) for i in range(3)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    tokenizer.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

In [10]:
joblib.dump([latent_dim, n_tokens, max_line_length, tokenizer], str(output_dir / 'metadata.pkl'))

['output_all_data_test_2/metadata.pkl']

# Training Model

In [11]:
training_model, lstm, lines, inputs, outputs = create_training_model(latent_dim, n_tokens)

filepath = str(output_dir / ("%s-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5" % latent_dim))
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger(str(output_dir / 'training_log.csv'), append=True, separator=',')

callbacks_list = [checkpoint, csv_logger]

training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=epochs, validation_split=.1, callbacks=callbacks_list)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 150868 samples, validate on 16764 samples
Epoch 1/10
  2048/150868 [..............................] - ETA: 39:21 - loss: 7.8077 - output_line_0_loss: 2.4026 - output_line_1_loss: 2.8941 - output_line_2_loss: 2.5110

KeyboardInterrupt: 

# Test Model

In [ ]:
generator = Generator(lstm, lines, tf_session, tokenizer, n_tokens, max_line_length)

In [ ]:
generator.generate_haiku()